## NFL Big Data Bowl

In [1]:
# imports
import pandas as pd
import numpy as np
from fpdf import FPDF

In [2]:
# User Inputs

# Which team you would like to scout out
comparison_team = "CHI"
# Options:
# ['DAL', 'TB', 'PHI', 'ATL', 'PIT', 'BUF', 'CAR', 'NYJ', 'CIN',
#  'MIN', 'SF', 'DET', 'JAX', 'HOU', 'SEA', 'IND', 'ARI', 'TEN',
#  'WAS', 'LAC', 'KC', 'CLE', 'MIA', 'NE', 'GB', 'NO', 'DEN', 'NYG',
#  'LA', 'CHI', 'BAL', 'LV']

comparison_threshold = 20 ## The threshold that determines if a team's tendancy is an anomoly: for example when set 
                          ## at 20 a team's metric would need to be 20% higher or lower than the baseline to be flagged

# This being true will display more dataframes within the notebook
qa_check = True ## True or False

In [3]:
## Set up PDF output
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 20)
title = comparison_team + " Defense Stunt Breakdown"
pdf.cell(w=0, h=20, txt=title, ln=1)

In [4]:
# get data
games = pd.read_csv("games.csv")
scouting = pd.read_csv("pffScoutingData.csv")
players = pd.read_csv("players.csv")
plays = pd.read_csv("plays.csv")
week1 = pd.read_csv("week1.csv")
week2 = pd.read_csv("week2.csv")
week3 = pd.read_csv("week3.csv")
week4 = pd.read_csv("week4.csv")
week5 = pd.read_csv("week5.csv")
week6 = pd.read_csv("week6.csv")
week7 = pd.read_csv("week7.csv")
week8 = pd.read_csv("week8.csv")

In [5]:
# Combine all position data into one dataset
all_weeks = week1.append(week2, ignore_index = True)
all_weeks = all_weeks.append(week3, ignore_index = True)
all_weeks = all_weeks.append(week4, ignore_index = True)
all_weeks = all_weeks.append(week5, ignore_index = True)
all_weeks = all_weeks.append(week6, ignore_index = True)
all_weeks = all_weeks.append(week7, ignore_index = True)
all_weeks = all_weeks.append(week8, ignore_index = True)

In [6]:
if qa_check:
    print(all_weeks)

             gameId  playId    nflId  frameId                     time  \
0        2021090900      97  25511.0        1  2021-09-10T00:26:31.100   
1        2021090900      97  25511.0        2  2021-09-10T00:26:31.200   
2        2021090900      97  25511.0        3  2021-09-10T00:26:31.300   
3        2021090900      97  25511.0        4  2021-09-10T00:26:31.400   
4        2021090900      97  25511.0        5  2021-09-10T00:26:31.500   
...             ...     ...      ...      ...                      ...   
8314173  2021110100    4433      NaN       54  2021-11-02T03:20:26.500   
8314174  2021110100    4433      NaN       55  2021-11-02T03:20:26.600   
8314175  2021110100    4433      NaN       56  2021-11-02T03:20:26.700   
8314176  2021110100    4433      NaN       57  2021-11-02T03:20:26.800   
8314177  2021110100    4433      NaN       58  2021-11-02T03:20:26.900   

         jerseyNumber      team playDirection      x      y     s     a   dis  \
0                12.0        T

In [7]:
# Find when the ball was snapped on each play to be able to determine 1.5 seconds of play time
ball_snap_df = all_weeks[(all_weeks['team'] == "football") & (all_weeks['event'] == "ball_snap")]
ball_snap_df = ball_snap_df[['gameId', 'playId', 'frameId']].reset_index(drop=True)
ball_snap_df.rename(columns={'frameId':'snapFrame'}, inplace=True)
if qa_check:
    print(ball_snap_df)

          gameId  playId  snapFrame
0     2021090900      97          6
1     2021090900     137          7
2     2021090900     187          6
3     2021090900     282          6
4     2021090900     349          7
...          ...     ...        ...
8527  2021110100    4310          7
8528  2021110100    4363          7
8529  2021110100    4392          7
8530  2021110100    4411          7
8531  2021110100    4433          7

[8532 rows x 3 columns]


In [8]:
# Combine position data with ball snap
all_weeks = pd.merge(all_weeks, ball_snap_df, how = "left", on = ["gameId", "playId"])
if qa_check:
    print(all_weeks)

             gameId  playId    nflId  frameId                     time  \
0        2021090900      97  25511.0        1  2021-09-10T00:26:31.100   
1        2021090900      97  25511.0        2  2021-09-10T00:26:31.200   
2        2021090900      97  25511.0        3  2021-09-10T00:26:31.300   
3        2021090900      97  25511.0        4  2021-09-10T00:26:31.400   
4        2021090900      97  25511.0        5  2021-09-10T00:26:31.500   
...             ...     ...      ...      ...                      ...   
8314173  2021110100    4433      NaN       54  2021-11-02T03:20:26.500   
8314174  2021110100    4433      NaN       55  2021-11-02T03:20:26.600   
8314175  2021110100    4433      NaN       56  2021-11-02T03:20:26.700   
8314176  2021110100    4433      NaN       57  2021-11-02T03:20:26.800   
8314177  2021110100    4433      NaN       58  2021-11-02T03:20:26.900   

         jerseyNumber      team playDirection      x      y     s     a   dis  \
0                12.0        T

In [ ]:
# This whole section is to ID the stunts
# It goes through every player rushing the passer in every game and identifies 
# any pass rushers that cross over one another in the first 1.5 seconds after the
# ball is snapped. If any two players do this the play is flagged as stunt = 1

game_list = all_weeks['gameId'].unique().tolist()

for game in game_list:
    one_game = all_weeks[(all_weeks['gameId'] == game)]
    play_list = one_game['playId'].unique().tolist()

    for play in play_list:
        
        one_play = one_game[(one_game['playId'] == play)]
        one_play = pd.merge(one_play, scouting[['gameId', 'playId', 'nflId', 'pff_role']], how = "left", on = ["gameId", "playId", "nflId"])

        one_play = one_play[one_play['pff_role']=="Pass Rush"]
        one_play_15frames = one_play[(one_play['frameId'] == one_play['snapFrame']) | (one_play['frameId'] == one_play['snapFrame']+14)]

        pivot_column = one_play_15frames["nflId"].astype(int).astype(str) + "_" + one_play_15frames["frameId"].astype(str)
        pivot_column = pivot_column.to_frame()
        pivot_column.rename(columns={0:'pivotColumn'}, inplace=True)

        one_play_15frames = one_play_15frames.join(pivot_column)

        pivoted = pd.pivot(one_play_15frames, index=['gameId','playId'], columns = 'pivotColumn', values = 'y')

        player_list = one_play_15frames['nflId'].astype(int).astype(str).unique().tolist()
        frame_list = one_play_15frames['frameId'].astype(int).astype(str).unique().tolist()
        
        if len(frame_list) < 2:
            frame_list = ['0', '0']

        stunt = 0
        try:
            for i in player_list:
                for j in player_list:
                    i0 = i+'_'+frame_list[0]
                    i1 = i+'_'+frame_list[1]
                    j0 = j+'_'+frame_list[0]
                    j1 = j+'_'+frame_list[1]
                    diff0 = pivoted.iloc[0][i0] - pivoted.iloc[0][j0]
                    diff1 = pivoted.iloc[0][i1] - pivoted.iloc[0][j1]

                    if (diff0 > 0 and diff1 <= 0) or (diff0 < 0 and diff1 >= 0):
                        stunt = 1
                        break

                if stunt == 1:
                    break

            data = {'gameId': [game], 'playId': [play], 'stunt': [stunt], 'num_rush': [len(player_list)]}
            
        except:
            data = {'gameId': [game], 'playId': [play], 'stunt': [None], 'num_rush': [None]}
            print('weird play')
            
        play_df = pd.DataFrame(data=data)

        try:
            stunt_outcome_df = stunt_outcome_df.append(play_df, ignore_index = True)
        except:
            stunt_outcome_df = play_df

stunt_outcome_df['stunt'] = pd.to_numeric(stunt_outcome_df['stunt'])

weird play


In [ ]:
if qa_check:
    print(stunt_outcome_df)

In [ ]:
### Find out if play ended in a pressure for future use (pressure is any combination of QB sack, hit or hurry)

In [ ]:
sack_df = scouting.groupby(['gameId', 'playId'])['pff_sack'].max().to_frame()
hit_df = scouting.groupby(['gameId', 'playId'])['pff_hit'].max().to_frame()
hurry_df = scouting.groupby(['gameId', 'playId'])['pff_hurry'].max().to_frame()

In [ ]:
pressure_df = pd.merge(sack_df, hit_df, how = "outer", on = ["gameId", "playId"])

In [ ]:
pressure_df = pd.merge(pressure_df, hurry_df, how = "outer", on = ["gameId", "playId"])

In [ ]:
pressure_df['pressure'] = [1 if x >= 1 else 0 for x in pressure_df['pff_sack']+pressure_df['pff_hit']+pressure_df['pff_hurry']]

In [ ]:
if qa_check:
    print(pressure_df)

In [ ]:
# Combine stunt ID data with pressure data
stunt_pressure_df = pd.merge(stunt_outcome_df, pressure_df, how = "left", on = ["gameId", "playId"])
if qa_check:
    print(stunt_pressure_df)

In [ ]:
# Add all play information for summary statistics
stunt_pressure_play_df = pd.merge(stunt_pressure_df, plays, how = "left", on = ["gameId", "playId"])
if qa_check:
    print(stunt_pressure_play_df)

In [ ]:
## See what situations stunts occur most often in

In [ ]:
stunt_pressure_play_df['to_go_str'] = ["0-3" if (x <= 3) else "3-7" if (x > 3 and x <= 7) else "7-10" if (x > 7 and x <= 10) else "10+" for x in stunt_pressure_play_df['yardsToGo']]

In [ ]:
stunt_filtered_df = stunt_pressure_play_df[stunt_pressure_play_df['down']>0]

In [ ]:
team_stunt_down_dist_df = stunt_filtered_df[['defensiveTeam', 'down', 'to_go_str', 'stunt']].groupby(['defensiveTeam', 'down', 'to_go_str']).mean()
team_stunt_down_dist_df = team_stunt_down_dist_df.reset_index()

In [ ]:
team_stunt_down_df = stunt_filtered_df[['defensiveTeam', 'down', 'stunt']].groupby(['defensiveTeam', 'down']).mean()
team_stunt_down_df['to_go_str']="all"
team_stunt_down_df = team_stunt_down_df.reset_index()
team_stunt_down_df = team_stunt_down_df[["defensiveTeam", "down", "to_go_str", "stunt"]]

In [ ]:
team_stunt_dist_df = stunt_filtered_df[['defensiveTeam', 'to_go_str', 'stunt']].groupby(['defensiveTeam', 'to_go_str']).mean()
team_stunt_dist_df['down']="all"
team_stunt_dist_df = team_stunt_dist_df.reset_index()
team_stunt_dist_df = team_stunt_dist_df[["defensiveTeam", "down", "to_go_str", "stunt"]]

In [ ]:
team_stunt_df = stunt_filtered_df[['defensiveTeam', 'stunt']].groupby(['defensiveTeam']).mean()
team_stunt_df['to_go_str']="all"
team_stunt_df['down']="all"
team_stunt_df = team_stunt_df.reset_index()
team_stunt_df = team_stunt_df[["defensiveTeam", "down", "to_go_str", "stunt"]]

In [ ]:
teams_stunts_all_df = team_stunt_down_df.append(team_stunt_dist_df, ignore_index = True).append(team_stunt_df, ignore_index = True)
teams_stunts_all_df = teams_stunts_all_df.sort_values(by = ['defensiveTeam', 'down', 'to_go_str']).reset_index(drop=True)
if qa_check:
    print(teams_stunts_all_df)

In [ ]:
nfl_stunt_down_dist_df = stunt_filtered_df[['down', 'to_go_str', 'stunt']].groupby(['down', 'to_go_str']).mean()
nfl_stunt_down_dist_df["league"] = "NFL"
nfl_stunt_down_dist_df = nfl_stunt_down_dist_df.reset_index()
nfl_stunt_down_dist_df = nfl_stunt_down_dist_df[["league", "down", "to_go_str", "stunt"]]
nfl_stunt_down_dist_df.rename(columns={'stunt':'NFLstunt'}, inplace=True)

In [ ]:
nfl_stunt_down_df = stunt_filtered_df[['down', 'stunt']].groupby(['down']).mean()
nfl_stunt_down_df["league"] = "NFL"
nfl_stunt_down_df["to_go_str"] = "all"
nfl_stunt_down_df = nfl_stunt_down_df.reset_index()
nfl_stunt_down_df = nfl_stunt_down_df[["league", "down", "to_go_str", "stunt"]]
nfl_stunt_down_df.rename(columns={'stunt':'NFLstunt'}, inplace=True)

In [ ]:
nfl_stunt_dist_df = stunt_filtered_df[['to_go_str', 'stunt']].groupby(['to_go_str']).mean()
nfl_stunt_dist_df["league"] = "NFL"
nfl_stunt_dist_df['down']="all"
nfl_stunt_dist_df = nfl_stunt_dist_df.reset_index()
nfl_stunt_dist_df = nfl_stunt_dist_df[["league", "down", "to_go_str", "stunt"]]
nfl_stunt_dist_df.rename(columns={'stunt':'NFLstunt'}, inplace=True)

In [ ]:
nfl_stunt = stunt_filtered_df[['stunt']].mean()
data = {'league': ["NFL"], 'down': ['all'], 'to_go_str': ["all"], 'NFLstunt': [nfl_stunt[0]]}
nfl_stunt_df = pd.DataFrame(data=data)

In [ ]:
nfl_stunts_all_df = nfl_stunt_down_df.append(nfl_stunt_dist_df, ignore_index = True).append(nfl_stunt_df, ignore_index = True)
nfl_stunts_all_df = nfl_stunts_all_df.sort_values(by = ['down', 'to_go_str']).reset_index(drop=True)
if qa_check:
    print(nfl_stunts_all_df)

In [ ]:
### Combine

team_comp_df = pd.merge(nfl_stunts_all_df, teams_stunts_all_df[teams_stunts_all_df['defensiveTeam'] == comparison_team], how = "left", on = ["down", "to_go_str"])
team_comp_df['defensiveTeam'] = team_comp_df['defensiveTeam'].fillna(comparison_team)
team_comp_df['stunt'] = team_comp_df['stunt'].fillna(0)
if qa_check:
    print(team_comp_df)

In [ ]:
nfl_stunt_total = team_comp_df[(team_comp_df['down'] == "all") & (team_comp_df['to_go_str'] == "all")]['NFLstunt'].tolist()[0]
team_stunt_total = team_comp_df[(team_comp_df['down'] == "all") & (team_comp_df['to_go_str'] == "all")]['stunt'].tolist()[0]


## Add overall stunt tendency to pdf output
pdf.set_font('Arial', 'B', 9)
pdf.cell(w=65, h=10, txt=comparison_team+' percentage of snaps with stunt:', border=1, ln=0, align='C')
pdf.cell(w=20, h=10, txt='', border=0, ln=0, align='C')
pdf.cell(w=65, h=10, txt='NFL Avg percentage of snaps with stunt:', border=1, ln=1, align='C')
pdf.cell(w=65, h=10, txt=str(100*(round(team_stunt_total,4)))+'%', border=1, ln=0, align='C')
pdf.cell(w=20, h=10, txt='', border=0, ln=0, align='C')
pdf.cell(w=65, h=10, txt=str(100*(round(nfl_stunt_total,4)))+'%', border=1, ln=1, align='C')
pdf.cell(w=30, h=10, txt='', border=0, ln=1, align='C')


In [ ]:
team_comp_df['stuntIndex'] = (1+(team_comp_df['stunt']-team_comp_df['NFLstunt'])/team_comp_df['NFLstunt'])*100
if qa_check:
    print(team_comp_df)

In [ ]:
## Situational stunt tendancy
pdf.set_font('Arial', 'B', 14)
pdf.cell(w=0, h=10, txt="Stunt Rate by Down/Distance", ln=1)
pdf.set_font('Arial', '', 8)

high_df = team_comp_df[team_comp_df['stuntIndex'] > 100 + comparison_threshold].reset_index(drop=True)
if high_df.shape[0] == 0:
    high_str = comparison_team+" does not have a significantly higher stunt rate than the league average in any down or distance situation"
    pdf.cell(w=0, h=5, txt=high_str, ln=1)
    print(high_str)
else:
    for i in range(high_df.shape[0]):
        high_str = "For down = "+high_df['down'][i]+" and yards to go = "+high_df['to_go_str'][i]+" "+comparison_team+" performs a stunt on "+round(100*high_df['stunt'][i], 2)+"% of plays compared to the league average of "+round(100*high_df['NFLstunt'][i], 2)+"%"
        pdf.cell(w=0, h=5, txt=high_str, ln=1)
        print(high_str)


In [ ]:
## Situational stunt tendancy
low_df = team_comp_df[team_comp_df['stuntIndex'] < 100 - comparison_threshold].reset_index(drop=True)
if low_df.shape[0] == 0:
    low_str = comparison_team+" does not have a significantly lower stunt rate than the league average in any down or distance situation"
    pdf.cell(w=0, h=5, txt=low_str, ln=1)
    print(low_str)
else:
    for i in range(low_df.shape[0]):
        low_str = "For down = "+str(low_df['down'][i])+" and yards to go = "+low_df['to_go_str'][i]+" "+comparison_team+" performs a stunt on "+str(round(100*low_df['stunt'][i], 2))+"% of plays compared to the league average of "+str(round(100*low_df['NFLstunt'][i], 2))+"%"
        pdf.cell(w=0, h=5, txt=low_str, ln=1)
        print(low_str)
        
pdf.cell(w=30, h=10, txt='', border=0, ln=1, align='C')

In [ ]:
nfl_stunts_full_df = nfl_stunts_all_df.append(nfl_stunt_down_dist_df, ignore_index = True)
nfl_stunts_full_df = nfl_stunts_full_df.sort_values(by = ['down', 'to_go_str']).reset_index(drop=True)
nfl_stunts_full_df.rename(columns={'league':'League','NFLstunt':'Stunt Percentage', 'down':'Down', 'to_go_str':'Yards to Go'}, inplace=True)

In [ ]:
team_stunts_full_df = teams_stunts_all_df.append(team_stunt_down_dist_df, ignore_index = True)
team_stunts_full_df = team_stunts_full_df.sort_values(by = ['defensiveTeam', 'down', 'to_go_str']).reset_index(drop=True)
team_stunts_full_df.rename(columns={'defensiveTeam':'Team','stunt':'Stunt Percentage', 'down':'Down', 'to_go_str':'Yards to Go'}, inplace=True)

In [ ]:
all_stunt_sit_df = team_stunts_full_df[team_stunts_full_df['Team'] == comparison_team].reset_index(drop=True)

## add stunt table to pdf
df = all_stunt_sit_df

pdf.set_font('Arial', 'B', 8)
pdf.cell(w=30, h=10, txt='Team', border=1, ln=0, align='C')
pdf.cell(w=30, h=10, txt='Down', border=1, ln=0, align='C')
pdf.cell(w=30, h=10, txt='Yards to Go', border=1, ln=0, align='C')
pdf.cell(w=30, h=10, txt='Stunt Percentage', border=1, ln=1, align='C')
# Table contents
pdf.set_font('Arial', '', 6)
for i in range(0, len(df)):
    pdf.cell(w=30, h=5, 
             txt=df['Team'].iloc[i], 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=str(df['Down'].iloc[i]), 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=df['Yards to Go'].iloc[i], 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=str(100*(round(df['Stunt Percentage'].iloc[i],4)))+'%', 
             border=1, ln=1, align='C')

pdf.cell(w=30, h=10, txt='', border=0, ln=1, align='C')    

In [ ]:
print(comparison_team, "'s average stunt percentage in all situations is the following:")
all_stunt_sit_df

In [ ]:
print("The NFL's average stunt percentage in all situations is the following:")
nfl_stunts_full_df

In [ ]:
### See what situations stunts are most effective in creating pressure compared to no stunt

In [ ]:
team_no_stunt_df = stunt_filtered_df[stunt_filtered_df['stunt']==0]
team_yes_stunt_df = stunt_filtered_df[stunt_filtered_df['stunt']==1]

In [ ]:
no_s_team_pressure_df1 = team_no_stunt_df[['defensiveTeam', 'down', 'to_go_str', 'pressure']].groupby(['defensiveTeam', 'down', 'to_go_str']).mean()
no_s_team_pressure_df1 = no_s_team_pressure_df1.reset_index()

In [ ]:
no_s_team_pressure_df2 = team_no_stunt_df[['defensiveTeam', 'down', 'pressure']].groupby(['defensiveTeam', 'down']).mean()
no_s_team_pressure_df2['to_go_str']="all"
no_s_team_pressure_df2 = no_s_team_pressure_df2.reset_index()
no_s_team_pressure_df2 = no_s_team_pressure_df2[["defensiveTeam", "down", "to_go_str", "pressure"]]

In [ ]:
no_s_team_pressure_df3 = team_no_stunt_df[['defensiveTeam', 'to_go_str', 'pressure']].groupby(['defensiveTeam', 'to_go_str']).mean()
no_s_team_pressure_df3['down']="all"
no_s_team_pressure_df3 = no_s_team_pressure_df3.reset_index()
no_s_team_pressure_df3 = no_s_team_pressure_df3[["defensiveTeam", "down", "to_go_str", "pressure"]]

In [ ]:
no_s_team_pressure_df4 = team_no_stunt_df[['defensiveTeam', 'pressure']].groupby(['defensiveTeam']).mean()
no_s_team_pressure_df4['to_go_str']="all"
no_s_team_pressure_df4['down']="all"
no_s_team_pressure_df4 = no_s_team_pressure_df4.reset_index()
no_s_team_pressure_df4 = no_s_team_pressure_df4[["defensiveTeam", "down", "to_go_str", "pressure"]]

In [ ]:
no_s_team_pressure_dfall = no_s_team_pressure_df2.append(no_s_team_pressure_df3, ignore_index = True).append(no_s_team_pressure_df4, ignore_index = True)
no_s_team_pressure_dfall = no_s_team_pressure_dfall.sort_values(by = ['defensiveTeam', 'down', 'to_go_str']).reset_index(drop=True)
no_s_team_pressure_dfall = no_s_team_pressure_dfall.rename(columns={'defensiveTeam':'Team', 'down':'Down', 'to_go_str':'Yards to Go', 'pressure':'No Stunt Pressure %'})
if qa_check:
    print(no_s_team_pressure_dfall)

In [ ]:
yes_s_team_pressure_df1 = team_yes_stunt_df[['defensiveTeam', 'down', 'to_go_str', 'pressure']].groupby(['defensiveTeam', 'down', 'to_go_str']).mean()
yes_s_team_pressure_df1 = yes_s_team_pressure_df1.reset_index()

In [ ]:
yes_s_team_pressure_df2 = team_yes_stunt_df[['defensiveTeam', 'down', 'pressure']].groupby(['defensiveTeam', 'down']).mean()
yes_s_team_pressure_df2['to_go_str']="all"
yes_s_team_pressure_df2 = yes_s_team_pressure_df2.reset_index()
yes_s_team_pressure_df2 = yes_s_team_pressure_df2[["defensiveTeam", "down", "to_go_str", "pressure"]]

In [ ]:
yes_s_team_pressure_df3 = team_yes_stunt_df[['defensiveTeam', 'to_go_str', 'pressure']].groupby(['defensiveTeam', 'to_go_str']).mean()
yes_s_team_pressure_df3['down']="all"
yes_s_team_pressure_df3 = yes_s_team_pressure_df3.reset_index()
yes_s_team_pressure_df3 = yes_s_team_pressure_df3[["defensiveTeam", "down", "to_go_str", "pressure"]]

In [ ]:
yes_s_team_pressure_df4 = team_yes_stunt_df[['defensiveTeam', 'pressure']].groupby(['defensiveTeam']).mean()
yes_s_team_pressure_df4['to_go_str']="all"
yes_s_team_pressure_df4['down']="all"
yes_s_team_pressure_df4 = yes_s_team_pressure_df4.reset_index()
yes_s_team_pressure_df4 = yes_s_team_pressure_df4[["defensiveTeam", "down", "to_go_str", "pressure"]]

In [ ]:
yes_s_team_pressure_dfall = yes_s_team_pressure_df2.append(yes_s_team_pressure_df3, ignore_index = True).append(yes_s_team_pressure_df4, ignore_index = True)
yes_s_team_pressure_dfall = yes_s_team_pressure_dfall.sort_values(by = ['defensiveTeam', 'down', 'to_go_str']).reset_index(drop=True)
yes_s_team_pressure_dfall = yes_s_team_pressure_dfall.rename(columns={'defensiveTeam':'Team', 'down':'Down', 'to_go_str':'Yards to Go', 'pressure':'Stunt Pressure %'})
if qa_check:
    print(yes_s_team_pressure_dfall)

In [ ]:
merged_pressure_df = pd.merge(no_s_team_pressure_dfall, yes_s_team_pressure_dfall, how = "left", on = ["Team", "Down", "Yards to Go"])
merged_pressure_df = merged_pressure_df[merged_pressure_df["Team"] == comparison_team].fillna(0).reset_index(drop=True)
if qa_check:
    print(merged_pressure_df)

In [ ]:
merged_pressure_df['stuntIndex'] = (1+(merged_pressure_df['Stunt Pressure %']-merged_pressure_df['No Stunt Pressure %'])/merged_pressure_df['No Stunt Pressure %'])*100
if qa_check:
    print(merged_pressure_df)

In [ ]:
## Add pressure info to PDF

pdf.set_font('Arial', 'B', 14)
pdf.cell(w=0, h=10, txt="Pressure Rates when stunting vs non-stunt plays", ln=1)
pdf.set_font('Arial', '', 8)

high_press_df = merged_pressure_df[merged_pressure_df['stuntIndex'] > 100 + comparison_threshold].reset_index(drop=True)
if high_press_df.shape[0] == 0:
    high_str = comparison_team+"does not have a significantly higher pressure rate when stunting than not in any major situation"
    pdf.cell(w=0, h=5, txt=high_str, ln=1)
    print(high_str)
else:
    for i in range(high_press_df.shape[0]):
        high_str = "For down = "+str(high_press_df['Down'][i])+" and yards to go = "+high_press_df['Yards to Go'][i]+" "+comparison_team+" gets pressure when performing a stunt on "+str(round(100*high_press_df['Stunt Pressure %'][i], 2))+"% of plays compared to "+str(round(100*high_press_df['No Stunt Pressure %'][i], 2))+"% on plays with no stunt"
        pdf.cell(w=0, h=5, txt=high_str, ln=1)
        print(high_str)

In [ ]:
## Add pressure info to PDF

low_press_df = merged_pressure_df[merged_pressure_df['stuntIndex'] < 100 - comparison_threshold].reset_index(drop=True)
if low_press_df.shape[0] == 0:
    low_str = comparison_team, "does not have a significantly lower pressure rate when stunting than not in any major situation"
    print(low_str)
    pdf.cell(w=0, h=5, txt=low_str, ln=1)
else:
    for i in range(low_press_df.shape[0]):
        low_str = "For down = "+str(low_press_df['Down'][i])+" and yards to go = "+low_press_df['Yards to Go'][i]+" "+comparison_team+" gets pressure when performing a stunt on "+str(round(100*low_press_df['Stunt Pressure %'][i], 2))+"% of plays compared to "+str(round(100*low_press_df['No Stunt Pressure %'][i], 2))+"% on plays with no stunt"
        print(low_str)
        pdf.cell(w=0, h=5, txt=low_str, ln=1)
        
pdf.cell(w=30, h=10, txt='', border=0, ln=1, align='C')   

In [ ]:
## Add pressure info to PDF

df = merged_pressure_df

pdf.set_font('Arial', 'B', 8)
pdf.cell(w=30, h=10, txt='Team', border=1, ln=0, align='C')
pdf.cell(w=30, h=10, txt='Down', border=1, ln=0, align='C')
pdf.cell(w=30, h=10, txt='Yards to Go', border=1, ln=0, align='C')
pdf.cell(w=30, h=10, txt='No Stunt Pressure %', border=1, ln=0, align='C')
pdf.cell(w=30, h=10, txt='Stunt Pressure %', border=1, ln=1, align='C')
# Table contents
pdf.set_font('Arial', '', 6)
for i in range(0, len(df)):
    pdf.cell(w=30, h=5, 
             txt=df['Team'].iloc[i], 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=str(df['Down'].iloc[i]), 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=df['Yards to Go'].iloc[i], 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=str(100*(round(df['No Stunt Pressure %'].iloc[i],4)))+'%', 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=str(100*(round(df['Stunt Pressure %'].iloc[i],4)))+'%', 
             border=1, ln=1, align='C')

pdf.cell(w=30, h=10, txt='', border=0, ln=1, align='C')    

In [ ]:
### Stunts by defensive personel

team_formation_df = stunt_filtered_df[stunt_filtered_df['defensiveTeam'] == comparison_team]
team_formation_stunt_df = team_formation_df[['defensiveTeam', 'personnelD', 'stunt']].groupby(['defensiveTeam', 'personnelD']).mean().reset_index()
team_formation_count_df = team_formation_df[['defensiveTeam', 'personnelD', 'stunt']].groupby(['defensiveTeam', 'personnelD']).count().rename(columns={'stunt':'count'}).reset_index()
team_formation_summary_df = pd.merge(team_formation_stunt_df, team_formation_count_df, how = "left", on = ["defensiveTeam", "personnelD"])
print(comparison_team, "'s stunt tendancies by defensive personnnel:")
team_formation_summary_df = team_formation_summary_df.rename(columns={'defensiveTeam':'Team', 'personnelD':'Defensive Personnel', 'stunt':'Stunt Percentage', 'count':'Total Play Count'})
team_formation_summary_df

In [ ]:
### Stunts by defensive personel

team_formation_df2 = stunt_filtered_df[stunt_filtered_df['defensiveTeam'] == comparison_team]
team_formation_stunt_df2 = team_formation_df2[['defensiveTeam', 'personnelO', 'stunt']].groupby(['defensiveTeam', 'personnelO']).mean().reset_index()
team_formation_count_df2 = team_formation_df2[['defensiveTeam', 'personnelO', 'stunt']].groupby(['defensiveTeam', 'personnelO']).count().rename(columns={'stunt':'count'}).reset_index()
team_formation_summary_df2 = pd.merge(team_formation_stunt_df2, team_formation_count_df2, how = "left", on = ["defensiveTeam", "personnelO"])
print(comparison_team, "'s stunt tendancies by opponent's offensive personnnel:")
team_formation_summary_df2 = team_formation_summary_df2.rename(columns={'defensiveTeam':'Team', 'personnelO':'Offensive Personnel', 'stunt':'Stunt Percentage', 'count':'Total Play Count'})
team_formation_summary_df2

In [ ]:
pdf.set_font('Arial', 'B', 14)
pdf.cell(w=0, h=10, txt="Stunt rate by offensive personnel faced", ln=1)
pdf.set_font('Arial', '', 10)

df = team_formation_summary_df2

# Table Header
pdf.set_font('Arial', 'B', 8)
pdf.cell(w=30, h=8, txt='Team', border=1, ln=0, align='C')
pdf.cell(w=30, h=8, txt='Offensive Personnel', border=1, ln=0, align='C')
pdf.cell(w=30, h=8, txt='Stunt Percentage', border=1, ln=0, align='C')
pdf.cell(w=30, h=8, txt='Total Play Count', border=1, ln=1, align='C')
# Table contents
pdf.set_font('Arial', '', 6)
for i in range(0, len(df)):
    pdf.cell(w=30, h=5, 
             txt=df['Team'].iloc[i], 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=df['Offensive Personnel'].iloc[i], 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=str(100*(round(df['Stunt Percentage'].iloc[i],4)))+'%', 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=df['Total Play Count'].iloc[i].astype(str), 
             border=1, ln=1, align='C')
    
pdf.cell(w=30, h=10, txt='', border=0, ln=1, align='C')  

In [ ]:
pdf.set_font('Arial', 'B', 14)
pdf.cell(w=0, h=10, txt="Stunt rate by defensive personnel", ln=1)
pdf.set_font('Arial', '', 10)

df = team_formation_summary_df

# Table Header
pdf.set_font('Arial', 'B', 8)
pdf.cell(w=30, h=8, txt='Team', border=1, ln=0, align='C')
pdf.cell(w=30, h=8, txt='Defensive Personnel', border=1, ln=0, align='C')
pdf.cell(w=30, h=8, txt='Stunt Percentage', border=1, ln=0, align='C')
pdf.cell(w=30, h=8, txt='Total Play Count', border=1, ln=1, align='C')
# Table contents
pdf.set_font('Arial', '', 6)
for i in range(0, len(df)):
    pdf.cell(w=30, h=5, 
             txt=df['Team'].iloc[i], 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=df['Defensive Personnel'].iloc[i], 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=str(100*(round(df['Stunt Percentage'].iloc[i],4)))+'%', 
             border=1, ln=0, align='C')
    pdf.cell(w=30, h=5, 
             txt=df['Total Play Count'].iloc[i].astype(str), 
             border=1, ln=1, align='C')
    
    
pdf.output(str(comparison_team+'output.pdf'))